# Dyson V2 Deposit & Premium Simulation
此範例模擬 Dyson V2 的存款 (Deposit) 與對應的 Premium 計算流程，
並展示如何使用 Python 進行簡單的狀態維護與測試。

In [406]:
import math
import numpy as np

########################################
# 初始參數設定
########################################

# 池子初始狀態
x = 100.0       # ETH
y = 200000.0    # USDC

# x = 1000.0  # ETH
# y = 1000.0  # USDC

# 目標當日可賣額度
daily_sell_target_eth = 200.0

# 協定計算需要的基礎變數
k = math.sqrt(x * y)  # sqrt(x*y)
w = daily_sell_target_eth * k  # 當日可賣流動性刻度

# 全域變數 q (今日累積使用的刻度)
q = 0.0  

basis = 0.5  # 假設年化波動率 50%
ln2 = 0.69314718055994530941723212145817656807550013436025525412068000949339362196 # ln(2)

### DysonV2 Premium 計算
- q_old = deposit前的池子累積 q 值（已賣額度）
- q_new = deposit後的池子累積 q 值（已賣額度）
- w = 池子可賣額度

$$
a = \frac{q_{old}}{w}, b = \frac{q_{new}}{w}
$$
$$
discount = ln(2) \times \frac{ \log_2(1+b) - \log_2(1+a) }{ b - a}
$$

$$
Premium
= \underbrace{0.4\,basis\,\sqrt{t}}_{\text{basic rate}}
\times
\underbrace{\bigl(\text{discount}\bigr)}_{\text{區段平均調整}}.
$$

In [407]:
import math


def calc_discount(a, b):
    if b <= a:
        raise ValueError("b should greater than a.")

    if a < 0 or b < 0:
        # 不預期情況: 此函式假設 a, b >= 0
        raise ValueError("a, b should be non-negative in this context.")

    # 如果 a > 0, 走正常公式:
    numerator = (math.log2(b + 1) - math.log2(a + 1)) * math.log(2)
    denom = b - a
    return numerator / denom

# --- 測試 ---
test_cases = [
    (0, 0.001),
    (0, 0.1),
    (0, 0.2),
    (0.2, 0.3),
    (0.2, 0.9),
    (0.9, 0.999),
    (0.9, 1.0),
]

for a, b in test_cases:
    d_log2 = calc_discount(a, b)
    print(f"[{a}, {b}] => discount={d_log2:.4f}")

[0, 0.001] => discount=0.9995
[0, 0.1] => discount=0.9531
[0, 0.2] => discount=0.9116
[0.2, 0.3] => discount=0.8004
[0.2, 0.9] => discount=0.6565
[0.9, 0.999] => discount=0.5131
[0.9, 1.0] => discount=0.5129


In [408]:
def calc_premium(q_old, q_new, basis, time_days):
    """
    最終 Premium = (0.4 * basis * sqrt(time)) * discount
    discount = (對應區間 [q_old, q_new] 的對數平均)
    """
    discount = calc_discount(q_old/w, q_new/w)

    # basic rate
    # t 若以天計算, 需先轉成年化, 看協定定義.
    # 這裡假設 time_days / 365.
    t_yr = time_days / 365.0
    basic_rate = 0.4 * basis * math.sqrt(t_yr)

    return basic_rate * discount

### DysonV2 deposit 公式
$$
Q = 4 \cdot (\sqrt{(x+input_0)(y+input_1)} - \sqrt{xy})^2
$$

In [409]:
def dyson_deposit_formula(balance0, balance1, inputAmt0, inputAmt1, duration):
    global q
    global x, y

    """
    依照 Dyson V2 公式計算:
    Q = 4 * ( sqrt((x+input0)*(y+input1)) - sqrt(x*y) )^2
    並回傳 (noteAmt0, noteAmt1, Q)
    """
    sqrt_xy = math.sqrt(balance0 * balance1)
    sqrt_xMyN = math.sqrt((balance0 + inputAmt0) * (balance1 + inputAmt1))
    diff = max(sqrt_xMyN - sqrt_xy, 0)
    Q = 4.0 * diff * diff

    # 判斷單邊比較: inputAmt0 / balance0 vs. inputAmt1 / balance1
    # 等價於 inputAmt0*balance1 > inputAmt1*balance0
    if (inputAmt0 * balance1) > (inputAmt1 * balance0):
        ratio = (inputAmt1 * balance0) / balance1 if balance1 > 0 else 0
        noteAmt0 = inputAmt0 + ratio
        if noteAmt0 == 0:
            noteAmt1 = 0
        else:
            noteAmt1 = Q / noteAmt0
    else:
        ratio = (inputAmt0 * balance1) / balance0 if balance0 > 0 else 0
        noteAmt1 = inputAmt1 + ratio
        if noteAmt1 == 0:
            noteAmt0 = 0
        else:
            noteAmt0 = Q / noteAmt1

    sqrt_Q = math.sqrt(Q)

    # 協定更新 q：
    q_old = q
    q_new = q + sqrt_Q  # 新的刻度
    print(f"q_old={q_old}, q_new={q_new}, sqrt_Q={sqrt_Q}")

    # 計算該筆投資的 Premium
    premium = calc_premium(q_old=q_old, q_new=q_new, basis=basis, time_days=duration)

    # 最後更新全域狀態
    q = q_new
    # 同時池子 x, y 也應更新 (模擬實際存入後的新餘額)
    x += inputAmt0
    y += inputAmt1

    return noteAmt0, noteAmt1, sqrt_Q, q_old, q_new, premium

In [410]:
def compute_withdraw_ratio(x, y, m, n):
    """
    計算使用者兌換比例 a，範圍應落在 [0, 1]。
    其中：
        x, y 為池子目前的 token0 和 token1 儲備
        m, n 為票據 note 中的 token0Amt, token1Amt
    """
    if m == 0 or n == 0:
        raise ValueError("m and n must be non-zero")

    a = 0.5 + x / (2 * m) - y / (2 * n)
    return max(0, min(a, 1))  # Clamp to [0, 1]

### Demo: 實際跑一筆 deposit

In [411]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.2f" % x)

# Create initial parameters DataFrame
initial_params = pd.DataFrame(
    {
        "Parameter": [
            "Pool ETH (x)",
            "Pool USDC (y)",
            "Daily Sell Target (ETH)",
            "K (sqrt(x*y))",
            "W (K * Daily Sell Target)",
            "Current q",
            "Basis (Annual Volatility)",
        ],
        "Value": [x, y, daily_sell_target_eth, k, w, q, basis],
    }
)


In [412]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.4f" % x)

def process_deposit(deposit_time_days, inputAmt0, inputAmt1):
    noteAmt0, noteAmt1, sqrt_Q, q_old, q_new, premium = dyson_deposit_formula(
        x, y, inputAmt0, inputAmt1, deposit_time_days
    )

    noteAmt0WithPremium = noteAmt0 * (1 + premium)
    noteAmt1WithPremium = noteAmt1 * (1 + premium)

    a = compute_withdraw_ratio(x, y, noteAmt0WithPremium, noteAmt1WithPremium)

    withdraw0 = noteAmt0WithPremium * a
    withdraw1 = noteAmt1WithPremium * (1 - a)

    return {
        "Days": deposit_time_days,
        "Input(ETH, USDC)": (inputAmt0, round(inputAmt1, 4)),
        "q": sqrt_Q,
        "q_old/w": q_old / w,
        "q_new/w": q_new / w,
        "Note(ETH, USDC)": (noteAmt0, round(noteAmt1, 4)),
        "Premium": premium,
        "NoteWithPremiun(ETH, USDC)": (
            noteAmt0WithPremium,
            noteAmt1WithPremium,
        ),
        "withdraw(ETH, USDC)": (withdraw0, round(withdraw1, 4)),
        "New Pool(ETH, USDC)": (x, y),
    }

In [413]:
print("\n=== Initial Parameters ===")
initial_params


=== Initial Parameters ===


,Parameter,Value
0,Pool ETH (x),100.0000
1,Pool USDC (y),200000.0000
2,Daily Sell Target (ETH),200.0000
3,K (sqrt(x*y)),4472.1360
4,W (K * Daily Sell Target),894427.1910
5,Current q,0.0000
6,Basis (Annual Volatility),0.5000


In [414]:
# Initialize results DataFrame
deposits_df = pd.DataFrame()

# First deposit
deposit1 = process_deposit(1, 10, 0)
deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit1])], ignore_index=True)

# Second deposit
deposit2 = process_deposit(1, 10, 0)
deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit2])], ignore_index=True)

print("=== Deposit Results ===")
deposits_df

q_old=0.0, q_new=436.5596096477002, sqrt_Q=436.5596096477002
q_old=436.5596096477002, q_new=853.6870611335526, sqrt_Q=417.12745148585236
=== Deposit Results ===


,Days,"Input(ETH, USDC)",q,q_old/w,q_new/w,"Note(ETH, USDC)",Premium,"NoteWithPremiun(ETH, USDC)","withdraw(ETH, USDC)","New Pool(ETH, USDC)"
0,1,"(10, 0)",436.5596,0.0000,0.0005,"(10.0, 19058.4293)",0.0105,"(10.104659245109278, 19257.893359691196)","(7.582108418300821, 4807.5857)","(110.0, 200000.0)"
1,1,"(10, 0)",417.1275,0.0005,0.0010,"(10.0, 17399.5311)",0.0105,"(10.104609334856981, 17581.546415600733)","(7.579491434738103, 4393.5867)","(120.0, 200000.0)"


In [415]:
(noteAmt0WithPremium, noteAmt1WithPremium) = deposits_df.iloc[0][
    "NoteWithPremiun(ETH, USDC)"
]
(user2Withdraw0, user2Withdraw1) = deposits_df.iloc[1]["withdraw(ETH, USDC)"]
a = compute_withdraw_ratio(x-user2Withdraw0, y-user2Withdraw1, noteAmt0WithPremium, noteAmt1WithPremium)

user1Withdraw0 = noteAmt0WithPremium * a
user1Withdraw1 = noteAmt1WithPremium * (1 - a)

print(f"After User2 withdraw, User1 can withdraw ETH: {user1Withdraw0}, USDC: {user1Withdraw1}")

After User2 withdraw, User1 can withdraw ETH: 9.945025031077098, USDC: 304.23773784136193


In [416]:
def to_q64_64(value: float) -> str:
    """
    將十進位浮點數轉換成 Q64.64 fixed point 十六進位表示。
    Q64.64 意味著乘以 2^64，再轉成十六進位。
    """
    q64_val = int(value * (1 << 64))  # value * 2^64
    return hex(q64_val)


# 範例：
print("ln(2):", to_q64_64(0.6931471805599453))  # ➜ 0xb17217f7d1cf79ac
print("pi:   ", to_q64_64(3.141592653589793))  # ➜ 0x3243f6a8885a308d
print("e:    ", to_q64_64(2.718281828459045))  # ➜ 0x2b7e151628aed2a6

ln(2): 0xb17217f7d1cf7800
pi:    0x3243f6a8885a30000
e:     0x2b7e151628aed2000
